## A simple model for demand and supply of publicly-provided services in a city

In [ ]:
from enum import Enum
import os.path

import numpy as np
import pandas as pd
import geopandas as gpd
import geopy, geopy.distance
import shapely
from sklearn import gaussian_process

from matplotlib import pyplot as plt 
import seaborn as sns
plt.rcParams['figure.figsize']= (20,14)

In [ ]:
## TODO: find way to put this into some global settings
import os
import sys
nb_dir = os.path.dirname(os.getcwd())
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

from references import common_cfg

In [ ]:
from src.models.city_items import AgeGroup, ServiceArea, ServiceType, SummaryNorm # enum classes for the model

In [ ]:
from src.models.services_supply import ServiceUnit, ServiceEvaluator, UnitFactory, SchoolFactory, LibraryFactory 
from src.models.services_supply import get_random_pos #TODO refactor this fun into common utils

from src.models.demand import DemandUnit, DemandUnitFactory

from src.models.process_tools import MappedPositionsFrame, ServiceValues, GridMaker, ValuesPlotter

In [ ]:
quicktest = [ServiceUnit(ServiceType.PoliceStation, 'Duomo', ageDiffusionIn=None), 
        ServiceUnit(ServiceType.PoliceStation, 'Ripamonti', 
                    position=geopy.Point(45.43, 9.201), ageDiffusionIn=None)]
ServiceEvaluator(quicktest).evaluate_services_at(MappedPositionsFrame(get_random_pos(4)))
del quicktest

In [ ]:
## Load scuole
scuoleFile =  '../data/processed/Milano_datiScuole.csv'
schoolLoader = UnitFactory.createLoader(ServiceType.School, scuoleFile)

# Initialise with a default lengthscale of 0.5 km
schoolUnits = schoolLoader.load(meanRadius=0.5)
schoolEval = ServiceEvaluator(schoolUnits)

In [ ]:
## Load scuole
bibliotecheFile =  '../data/processed/Milano_biblioteche.csv'
bibliotecheLoader = UnitFactory.createLoader(ServiceType.Library, bibliotecheFile)

# Initialise with a default lengthscale of 0.5 km
libraryUnits = bibliotecheLoader.load(meanRadius=0.5)

In [ ]:
allEvaluator = ServiceEvaluator(schoolUnits + libraryUnits)
testEvaluator = ServiceEvaluator(schoolUnits[0:10])

In [ ]:
def process_city_demand(cityName):
    assert cityName in common_cfg.cityList, 'Unrecognized city name %s' % cityName
    cityDemandFactory = DemandUnitFactory(cityName)
    unitsOut = cityDemandFactory.make_units_at_centroids()
    mappedPosDemand = cityDemandFactory.export_mapped_positions()
    return mappedPosDemand, unitsOut

mappedDemand,_ = process_city_demand('Milano')

In [ ]:
# call grid making to discretise service evaluation, this is an alternative to evaluating on the demand units
milanoGridMK = GridMaker({'quartieri':'../data/raw/Milano_specific/Milano_quartieri.geojson'}, gridStep=.5)

In [ ]:
# compute service levels
#valuesGrid = testEvaluator.evaluate_services_at(milanoGridMK.grid)
valuesGrid = testEvaluator.evaluate_services_at(mappedDemand)

In [ ]:
from scipy.interpolate import griddata
                
class ValuesPlotter:
    '''
    A class that plots various types of output from a UnitAggregator
    '''
    def __init__(self, serviceValues):
        assert isinstance(serviceValues, ServiceValues), 'ServiceValues class expected'
        self.values = serviceValues
        #self.ua = unitAggregatorIn
        
    def plot_locations(self):
        '''
        Plots the locations of the initialized ServiceValues'
        '''
        plotScales = self.ua.scale/np.mean(self.ua.scale)
        plt.figure()
        plt.scatter(self.ua.longitude, self.ua.latitude, s=plotScales)
        plt.axis('equal')
        plt.show()
        return None
    
        
    def plot_service_levels(self, servType, gridDensity=40):
        '''
        Plots a contour graph of the results for each ageGroup.
        '''
        assert isinstance(servType, ServiceType), 'ServiceType expected in input'
        
        for ageGroup, valuesSeries in self.values[servType].items():
            valuesArray = valuesSeries.values
            coordsList = list(zip(*valuesSeries.index.levels[1]))
            xPlot = coordsList[1]
            yPlot = coordsList[0]
            if np.count_nonzero(valuesArray) > 0:
                # grid the data using natural neighbour interpolation
                xi = np.linspace(min(xPlot), max(xPlot), gridDensity)
                yi = np.linspace(min(yPlot), max(yPlot), gridDensity)
                zi = griddata((xPlot, yPlot), valuesArray, (xi[None,:], yi[:,None]), 'linear')
                # clip to zero
                bNeg = ~np.isnan(zi) & (zi<0)
                #zi[bNeg] = 0
                plt.figure()
                plt.title(ageGroup)
                CS = plt.contourf(xi, yi, zi, 20)
                cbar = plt.colorbar(CS)
                cbar.ax.set_ylabel('Service level')
                plt.show()
            
        return None

In [ ]:
plotterNew = ValuesPlotter(valuesGrid)
plotterNew.plot_service_levels(ServiceType.School, gridDensity=200) # plots with griddata+contourf

In [ ]:
## WORK IN PROGRESS - not completed
class Aggregator:
    '''A class to aggregate provided values for mapped positions'''
    
    def __init__(self, serviceValues):
        assert isinstance(serviceValues, ServiceValues), 'ServiceValues class expected'
        self.sourceValues = serviceValues
        self.services = list(serviceValues.keys())
        self.quartieri = serviceValues.mappedPositions.index.levels[0]
    
    
    def aggregate_values(self):
        # initialize output
        averagedValues = {service: pd.DataFrame(np.zeros([len(self.quartieri), len(AgeGroup.all())]), 
                                 index=self.quartieri, columns=AgeGroup.all()) 
                 for service in self.services}
        
        for service, data in self.sourceValues.items():
            averagedValues[service] = data.groupby(common_cfg.IdQuartiereColName).mean()
        
        return averagedValues
       
    #for quartiere in quartieri:
     #       bThisQuartiere = (self.IDquartiere == quartiere)
            # loop over service types
      #      if not bThisQuartiere.any():
       #         print('Quartiere %s has no points in grid within it' % quartiere)
        #        leftOutZones.append(quartiere)
        #        continue
                
         #   for servType in self.serviceValues.keys():
                
                
          #      for ageGroup, valuesSeries in self.serviceValues[servType].items(): 
          #          quartiereValues = [valuesSeries[tuple(p)] for p in self.plotPoints[bThisQuartiere]]
          #          averagedValues[servType].loc[quartiere, ageGroup] = np.mean(quartiereValues)
                    

In [ ]:
zz = Aggregator(valuesGrid)
#zz.aggregate_values()
ww = zz.aggregate_values()
ww[ServiceType.Library]